In [ ]:

! pip install -q pytorch-lightning

In [ ]:
import pytorch_lightning as pl
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from pytorch_lightning import Trainer
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from torchmetrics import Accuracy

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class CryptoDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        X = self.df.drop(columns=['if_long']).iloc[idx].to_numpy()
        Y = self.df['if_long'].iloc[idx]
        tensor_x = torch.from_numpy(X).float()
        mean = torch.mean(tensor_x, dim=0)
        std = torch.std(tensor_x, dim=0)
        tensor_x = (tensor_x - mean) / std

        tensor_y = torch.tensor(Y).float()
        return tensor_x, tensor_y

In [ ]:
class CryptoDataModule(pl.LightningDataModule):
    def __init__(self, df: pd.DataFrame):
        super().__init__()
        self.df = df

    def prepare_data(self):
        # Split dataframe into train, validation, and test sets
        df_train, df_test = train_test_split(self.df, test_size=0.2, random_state=42)
        df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)
        
        self.train_set = CryptoDataset(df_train)
        self.val_set = CryptoDataset(df_val)
        self.test_set = CryptoDataset(df_test)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=64, num_workers=3)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=64, num_workers=3)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=64, num_workers=3)


In [ ]:


class CryptoClassification(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(136, 64)
        self.layer_2 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)
        self.accuracy = Accuracy(task='binary')
        
    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze(1)
        loss = F.binary_cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze(1)
        loss = F.binary_cross_entropy(y_hat, y)
        self.log('val_loss', loss)
        return {'val_loss': loss}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze(1)
        loss = F.binary_cross_entropy(y_hat, y)
        acc = self.accuracy(y_hat, y)
        self.log('accuracy', acc)
        # y_hat = y_hat.detach().cpu().numpy()
        # y_hat = (y_hat >= 0.5).astype(int)
        # accuracy = (y_hat == y).mean()
        # self.log('test_accuracy', accuracy)
        # self.log('test_loss', loss)
        return {'test_loss': loss}
    
    # def test_epoch_end(self, outputs):
    #     avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
    #     self.log('test_loss', avg_loss)
    #     y_hat = self(x)
    #     y_hat = y_hat.detach().cpu().numpy()
    #     y_hat = (y_hat >= 0.5).astype(int)
    #     accuracy = (y_hat == y).mean()
    #     self.log('test_accuracy', accuracy)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        return optimizer


In [ ]:
import os

os.path.exists("/content/drive/MyDrive/BTCUSDT-15m.csv")

True

In [ ]:
call_type = "long"
df = pd.read_csv("/content/drive/MyDrive/BTCUSDT-15m.csv")
# Drop unused columns
df.drop(["open_time", "close_time"], axis=1, inplace=True)
df.drop(["entered_before", "longs_win_after", "shorts_win_after"], axis=1, inplace=True)
columns_to_drop = ['open',
 'high',
 'low',
 'close',
 'volume',
 'quote_volume',
 'count',
 'taker_buy_volume',
 'taker_buy_quote_volume']

df.drop(columns_to_drop, axis=1, inplace=True)
if call_type == "long":
    df.drop("if_short", axis=1, inplace=True)
else:
    df.drop("if_long", axis=1, inplace=True)
    
df.replace(np.NaN, -500, inplace=True)
df.if_long.replace(-1, 0, inplace=True)
# df.if_short.replace(-1, 0, inplace=True)
print(df.shape)


(104802, 137)


In [ ]:
# create model and data_module object
model = CryptoClassification()
data_module = CryptoDataModule(df)

# create trainer with early stopping and model checkpointing
early_stop_callback = pl.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="min")
checkpoint_callback = pl.callbacks.ModelCheckpoint(filename='{epoch}-{val_loss:.2f}', save_top_k=1)
logger = TensorBoardLogger("lightning-logs", name="crypto")
trainer = Trainer(max_epochs=1, callbacks=[early_stop_callback, checkpoint_callback])

# train and test the model
trainer.fit(model, data_module)
trainer.test(model, datamodule=data_module)


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type           | Params
------------------------------------------------
0 | layer_1      | Linear         | 8.8 K 
1 | layer_2      | Linear         | 2.1 K 
2 | output_layer | Linear         | 33    
3 | accuracy     | BinaryAccuracy | 0     
------------------------------------------------
10.9 K    Trainable params
0         Non-trainable params
10.9 K    Total params
0.044     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy            0.5740661025047302
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'accuracy': 0.5740661025047302}]

In [ ]:
! tensorboard 